### Project Title : - Using Lung ultrasound images for building a reliable Point-of-care Covid-19 testing system

#### Multi Classifier with stratified sampling

In [ ]:
from google.colab import drive
drive.mount("/content/drive/")

Mounted at /content/drive/


In [ ]:
cd drive/MyDrive/

/content/drive/MyDrive


#### For 3800 images

#### Image Loading and Pre-processing

In [ ]:
#Importing the libraries

import pathlib
from pathlib import Path
from tensorflow.keras.preprocessing.image import load_img, img_to_array, smart_resize
import numpy as np
import tensorflow as tf
import cv2

In [ ]:
p = Path().cwd()
q = p/'stratify'

target_dict = {"normal" : 0, "covid" : 1 , "pneumonia" : 2}
images = [] # create an empty list
labels = [] # create an empty list  

for fldr in q.iterdir(): 
    current_label = fldr.name # get the name of the current label 
    print(current_label)  
    for img in fldr.iterdir():
        img = load_img(img,grayscale = True)
        img_array = img_to_array(img) # convert the image to an NumPy array 
        size = (198,197)
        img_array_resized = smart_resize(img_array,size)
        img_arr = img_array_resized.repeat(3,axis = -1)
        images.append(img_arr)# add reshaped image to list of images 
        labels.append(current_label)# add label for this image to the list of labels 

images = np.array(images) # convert the list of images to a NumPy array 
labels = np.array(labels) # convert the list of labels to a NumPy array

images = images.astype('float32') / 255
        

covid


/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/utils.py:107: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn('grayscale is deprecated. Please use '


normal
pneumonia


In [ ]:
from tensorflow.keras.utils import to_categorical

labels = np.asarray(list(map(lambda x: target_dict[x] if x in target_dict.keys() else x , labels))) # use the target_dict to convert the string labels to an array of integers
labels = to_categorical(labels)


####Splitting the dataset with stratification

In [ ]:
from sklearn.model_selection import train_test_split


#Splitting data into training,testing and validation
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.3, random_state=111,stratify = labels)

In [ ]:
print(X_train.shape)
print(y_train.shape)
print("---------")
print(X_test.shape)
print(y_test.shape)

(2310, 198, 197, 3)
(2310, 3)
---------
(990, 198, 197, 3)
(990, 3)


####Multi-classifier VGG16

In [ ]:
pretrained= tf.keras.applications.VGG16(include_top=False,
                   input_shape=(198,197,3), weights='imagenet')

58900480/58889256 [==============================] - 0s 0us/step


In [ ]:
inputs = tf.keras.Input(shape=(198,197,3))

x = pretrained(inputs)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(512)(x)
x = tf.keras.layers.Dropout(0.5)(x)
outputs = tf.keras.layers.Dense(3, activation="softmax")(x)

multi_vgg16 = tf.keras.Model(inputs, outputs)

multi_vgg16.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 198, 197, 3)]     0         
                                                                 
 vgg16 (Functional)          (None, 6, 6, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 18432)             0         
                                                                 
 dense (Dense)               (None, 512)               9437696   
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 3)                 1539      
                                                                 
Total params: 24,153,923
Trainable params: 24,153,923
Non-tra

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(featurewise_center=True,
                             featurewise_std_normalization=True)
  
# calculating the mean on the training dataset
datagen.fit(X_train)
  
# preparing iterators to scale images
train_iterator = datagen.flow(X_train,y_train, batch_size = 128)
test_iterator = datagen.flow(X_test,y_test, batch_size = 128)

In [ ]:
multi_vgg16.compile(loss="categorical_crossentropy",optimizer="Adam", metrics=["accuracy"])


In [ ]:
network = multi_vgg16.fit( train_iterator,steps_per_epoch = len(train_iterator),epochs= 5)

Epoch 1/5
19/19 [==============================] - 55s 1s/step - loss: 5.4118 - accuracy: 0.3779
Epoch 2/5
19/19 [==============================] - 24s 1s/step - loss: 0.9609 - accuracy: 0.5606
Epoch 3/5
19/19 [==============================] - 24s 1s/step - loss: 0.6396 - accuracy: 0.7177
Epoch 4/5
19/19 [==============================] - 24s 1s/step - loss: 0.3549 - accuracy: 0.8662
Epoch 5/5
19/19 [==============================] - 24s 1s/step - loss: 0.4980 - accuracy: 0.8221


In [ ]:
final_loss, final_acc = multi_vgg16.evaluate(test_iterator)
print("Test accuracy  : ",final_acc)

8/8 [==============================] - 9s 1s/step - loss: 0.2499 - accuracy: 0.9101
Test accuracy  :  0.9101009964942932


####For 5700 images

In [ ]:
p = Path().cwd()
q = p/'stratify'

target_dict = {"normal" : 0, "covid" : 1 , "pneumonia" : 2}
images = [] # create an empty list
labels = [] # create an empty list  

for fldr in q.iterdir(): 
    current_label = fldr.name # get the name of the current label 
    print(current_label)  
    for img in fldr.iterdir():
        img = load_img(img,grayscale = True)
        img_array = img_to_array(img) # convert the image to an NumPy array 
        size = (198,197)
        img_array_resized = smart_resize(img_array,size)
        img_arr = img_array_resized.repeat(3,axis = -1)
        images.append(img_arr)# add reshaped image to list of images 
        labels.append(current_label)# add label for this image to the list of labels 

images = np.array(images) # convert the list of images to a NumPy array 
labels = np.array(labels) # convert the list of labels to a NumPy array

images = images.astype('float32') / 255
        

covid


/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/utils.py:107: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn('grayscale is deprecated. Please use '


normal
pneumonia


In [ ]:
from tensorflow.keras.utils import to_categorical

labels = np.asarray(list(map(lambda x: target_dict[x] if x in target_dict.keys() else x , labels))) # use the target_dict to convert the string labels to an array of integers
labels = to_categorical(labels)


In [ ]:
from sklearn.model_selection import train_test_split


#Splitting data into training,testing and validation
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.3, random_state=111,stratify = labels)

In [ ]:
print(X_train.shape)
print(y_train.shape)
print("---------")
print(X_test.shape)
print(y_test.shape)

(4039, 198, 197, 3)
(4039, 3)
---------
(1731, 198, 197, 3)
(1731, 3)


#### Multi-classifier VGG16 with data augmentation

In [ ]:

data_augmentation = tf.keras.Sequential([ tf.keras.layers.RandomFlip("horizontal"),
                                      tf.keras.layers.RandomZoom(0.2)])

inputs = tf.keras.Input(shape=(198,197,3))
x = data_augmentation(inputs)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(512)(x)
x = tf.keras.layers.Dropout(0.5)(x)
outputs = tf.keras.layers.Dense(3, activation="softmax")(x)

multi_vgg16 = tf.keras.Model(inputs, outputs)

multi_vgg16.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 198, 197, 3)]     0         
                                                                 
 sequential_2 (Sequential)   (None, 198, 197, 3)       0         
                                                                 
 flatten_3 (Flatten)         (None, 117018)            0         
                                                                 
 dense_6 (Dense)             (None, 512)               59913728  
                                                                 
 dropout_3 (Dropout)         (None, 512)               0         
                                                                 
 dense_7 (Dense)             (None, 3)                 1539      
                                                                 
Total params: 59,915,267
Trainable params: 59,915,267
Non-t

In [ ]:
multi_vgg16.compile(loss="categorical_crossentropy",optimizer="Adam", metrics=["accuracy"])


In [ ]:
network = multi_vgg16.fit( train_iterator,steps_per_epoch = len(train_iterator),epochs= 3)

Epoch 1/3
32/32 [==============================] - 6s 146ms/step - loss: 108.2543 - accuracy: 0.6732
Epoch 2/3
32/32 [==============================] - 5s 161ms/step - loss: 51.4791 - accuracy: 0.7757
Epoch 3/3
32/32 [==============================] - 5s 147ms/step - loss: 31.7835 - accuracy: 0.8155


In [ ]:
final_loss, final_acc = multi_vgg16.evaluate(test_iterator)
print("Test accuracy  : ",final_acc)

14/14 [==============================] - 2s 136ms/step - loss: 20.0118 - accuracy: 0.8515
Test accuracy  :  0.851530909538269
